In [87]:
import os
import time
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains



In [88]:
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"

In [89]:
url = "https://www.bigbasket.com/pd/266564/taj-mahal-tea-500-g/"

In [96]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito")
chrome_options.add_argument(f'user_agent={user_agent}')
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("--disable-infobars")
driver = webdriver.Chrome(options=chrome_options)
driver.get(url)
wait = WebDriverWait(driver, 10)

In [97]:
# 1. Category, Sub-Category, and Product
div_category = driver.find_element(By.CSS_SELECTOR, "div._3moNK")
a_tags = div_category.find_elements(By.CSS_SELECTOR, "a._3WUR_._3bj9B.rippleEffect")[1:]
category, sub_category, product = [a_tag.text.replace(">", "").strip() for a_tag in a_tags]

# 2. SKU ID
sku_id = url.split("/")[-3]

# 3. Image
image_div = driver.find_element(By.CSS_SELECTOR, "div._2FbOx")
image = image_div.find_element(By.CSS_SELECTOR, "img._3oKVV").get_attribute("src")

# 4. Brand
brand_div = driver.find_element(By.CSS_SELECTOR, "div._2yfKw")
brand = brand_div.find_element(By.CSS_SELECTOR, "a._2zLWN._3bj9B.rippleEffect").text

# 5. SKU Name and SKU Size
sku_div = driver.find_element(By.CSS_SELECTOR, "div._2yfKw")
sku_string = sku_div.find_element(By.CSS_SELECTOR, "h1.GrE04").text
sku_name = sku_string.split(" - ")[0].strip()
sku_size = sku_string.split(",")[-1].strip()

# 6. MRP and Selling Price
price_div = driver.find_element(By.CSS_SELECTOR, "div#price")
try:
    mrp_element = price_div.find_element(By.CSS_SELECTOR, "td._2ifWF")
    mrp = mrp_element.text
except NoSuchElementException:
    mrp = "NA"
selling_price_element = price_div.find_element(By.CSS_SELECTOR, "td.IyLvo")
selling_price = selling_price_element.text

# 7. Out of Stock
try:
    driver.find_element(By.CSS_SELECTOR, "div._36qqs")
    in_stock = "No"
except NoSuchElementException:
    in_stock = "Yes"
out_of_stock = "Yes" if in_stock == "No" else "No"

# 8. Product Rating
try:
    product_rating_element = driver.find_element(By.CSS_SELECTOR, "div._2Ze34")
    product_rating = product_rating_element.text
except (NoSuchElementException, TimeoutException):
    product_rating = "NA"

# 9. Total Ratings and Reviews
try:
    review_div = driver.find_element(By.CSS_SELECTOR, "div._1AXTE")
    reviews_element = review_div.find_element(By.CSS_SELECTOR, "span[style='color: rgb(74, 74, 74); position: relative; top: 1px;']")
    total_reviews_ratings = reviews_element.text
except NoSuchElementException:
    total_reviews_ratings = "NA"

# Separate Total Ratings and Total Reviews
if total_reviews_ratings != "NA":
    total_ratings, total_reviews = total_reviews_ratings.split('&')
    total_ratings = total_ratings.strip()[:-8]  # remove " Ratings"
    total_reviews = total_reviews.strip()[:-8]  # remove " Reviews"
else:
    total_ratings = total_reviews = "NA"


# 10. Reviews
try:
    reviews_link_element = driver.find_element(By.CSS_SELECTOR, "a._1xG1d")
    reviews_link = reviews_link_element.get_attribute('href')
    driver.get(reviews_link)

    actions = ActionChains(driver)
    old_scroll_height = 0

    while True:
        actions.send_keys(Keys.END).perform()
        sleep(5)  # you may want to adjust this delay
        new_scroll_height = driver.execute_script("return document.documentElement.scrollHeight")
        
        if new_scroll_height == old_scroll_height:
            break
        else:
            old_scroll_height = new_scroll_height

    # Extract all reviews
    review_elements = driver.find_elements(By.CSS_SELECTOR, "div.zF-ya")
    reviews = [review_element.text for review_element in review_elements]
except NoSuchElementException:
    reviews = []


data = {
    "Category": category,
    "Sub-Category": sub_category,
    "Product": product,
    "SKU ID": sku_id,
    "Image": image,
    "Brand": brand,
    "SKU Name": sku_name,
    "SKU Size": sku_size,
    "MRP": mrp,
    "Selling Price": selling_price,
    "In Stock": in_stock,
    "Out of Stock": out_of_stock,
    "Product Rating": product_rating,
    "Total Ratings": total_ratings,
    "Total Reviews": total_reviews,
    "Reviews": reviews
}

driver.quit()

In [98]:
data

{'Category': 'BEVERAGES',
 'Sub-Category': 'TEA',
 'Product': 'LEAF & DUST TEA',
 'SKU ID': '266564',
 'Image': 'https://www.bigbasket.com/media/uploads/p/l/266564_11-taj-mahal-tea.jpg',
 'Brand': 'Taj Mahal',
 'SKU Name': 'Taj Mahal Tea, 500 g',
 'SKU Size': '500 g',
 'MRP': 'NA',
 'Selling Price': 'Rs 425',
 'In Stock': 'Yes',
 'Out of Stock': 'No',
 'Product Rating': '4.1',
 'Total Ratings': '2295',
 'Total Reviews': '17',
 'Reviews': ['I am using form long time tajmahal tea ist time The unsatisfactory item supplied tea contains a large amount of tea dust.',
  'quality issue',
  "The pack I ordered was sealed from outside, ie The cardboard packing looked intact, but the foil pack inside looked as if it had been cut open, and resealed! The tea was spilling outside the foil, be careful while ordering online. Check internal packing immediately, while it's still returnable.",
  'Tajmahal tea is wellknow for tea, i feel like got best deal 1kg for 750 INR',
  'Thank you seller ... Fast de